In [1]:
import cfscrape
from lxml import etree
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import re
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from bs4 import BeautifulSoup
from random import randint
from time import sleep
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go
from datetime import date
from itertools import cycle
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import pickle

## Getting links

In [3]:
chrome_options = webdriver.ChromeOptions()
chrome_options.headless=False
browser =  webdriver.Chrome('/Users/macbook/Downloads/Chrome Driver/chromedriver', chrome_options=chrome_options)

In [2]:
website = 'Globus'

In [152]:
website_links = ['https://www.globus-baumarkt.de/garten-freizeit/gartenbau-terrassenbau/holzfliesen/?p={}&ffFollowSearch=9640&o=8&n=100',
                'https://www.globus-baumarkt.de/garten-freizeit/gartenbau-terrassenbau/terrassendielen/holzterrassendielen/?p={}&ffFollowSearch=9640&o=8&n=100',
                'https://www.globus-baumarkt.de/holz-bauelemente/bauholz/?p={}&ffFollowSearch=9740&o=8&n=100',
                'https://www.globus-baumarkt.de/holz-bauelemente/laminat-parkett-vinyl/parkett/?p={}&ffFollowSearch=9640&o=8&n=100',
                'https://www.globus-baumarkt.de/holz-bauelemente/osb-verlegeplatten/?p={}&ffFollowSearch=9740&o=8&n=100'
                ]
pages = [1,1,4,1,1]

In [155]:
all_links = []
for idx, url_main in enumerate(website_links):
    
    sleep(2)
    pagination = pages[idx]
    
    for n in range(1,pagination+1):
        url = url_main.format(n)
        
        browser.get(url)
    
        path = '//*[@class="listing--container"]//a'
        page_links_raw = browser.find_elements_by_xpath(path)
        
        page_links=[]
        for item_url in page_links_raw:
            link = item_url.get_attribute('href')
            if link not in page_links:
                page_links.append(link)
    
        all_links.extend(page_links)
        print('\nSection:', idx+1, "| Page:", n, "| Products:", len(page_links))
        print('All Links', len(all_links))



Section: 1 | Page: 1 | Products: 10
All Links 10

Section: 2 | Page: 1 | Products: 34
All Links 44

Section: 3 | Page: 1 | Products: 101
All Links 145

Section: 3 | Page: 2 | Products: 101
All Links 246

Section: 3 | Page: 3 | Products: 101
All Links 347

Section: 3 | Page: 4 | Products: 42
All Links 389

Section: 4 | Page: 1 | Products: 20
All Links 409

Section: 5 | Page: 1 | Products: 8
All Links 417


In [22]:
with open('{}-RawLinks.pkl'.format(website), 'wb') as f:
    pickle.dump(all_links, f)

In [120]:
with open('{}-RawLinks.pkl'.format(website), 'rb') as f:
    page_links = pickle.load(f)

### Testing Options

In [176]:
browser =  webdriver.Chrome('/Users/macbook/Downloads/Chrome Driver/chromedriver', chrome_options=chrome_options)

In [156]:
page_links = all_links
all_links = []
for link in page_links:
    if link not in all_links and link != None:
        all_links.append(link)
len(all_links)

409

In [177]:
%%time
data = pd.DataFrame()
for i in range(0, len(all_links)): 
    url = all_links[i]
    
    if url in data['URL'].unique():
        continue
    else:
        pass
    
    browser.get(url)
    
    

    #sleep(1)
    path = '//*[@class="series--wrapper"]//select'
    selections = browser.find_elements_by_xpath(path)

    if len(selections)>1:
        path = '//*[@class="series--wrapper"]/div[1]//option'
        first_selections = browser.find_elements_by_xpath(path)

        first_opt=[]
        for opt in first_selections:
            link = opt.get_attribute('data-url')
            if link not in first_opt and link!=None and len(link)>0:
                first_opt.append(link)

        first_opt = ['https://www.globus-baumarkt.de' + x for x in first_opt]

        options_links = []       
        for link in first_opt:
            browser.get(link)
            sleep(1)

            path = '//*[@class="series--select"]//option'
            raw_options = browser.find_elements_by_xpath(path)


            for opt in raw_options:
                link = opt.get_attribute('data-url')
                if link not in options_links and link!=None and len(link)>0: 
                    options_links.append(link)

    elif len(selections)==1:
        path = '//*[@class="series--select"]//option'
        raw_options = browser.find_elements_by_xpath(path)
        options_links = [] 
        
        for opt in raw_options:
            link = opt.get_attribute('data-url')
            if link not in options_links and link!=None and len(link)>0: 
                options_links.append(link)
    
    else:
        options_links = []

    print('\nLink #', i, '| Number of options:', len(options_links))

    # Getting Options
    if len(options_links)>1:
        options_links = ['https://www.globus-baumarkt.de' + x for x in options_links]
        section_data = pd.DataFrame()
        
        for link in options_links:
            if link in data['URL'].unique():
                continue
            browser.get(link)
            sleep(1)
            page_data = get_page_data()
            page_data = cols_dedup(page_data)
            section_data = pd.concat([section_data, page_data], ignore_index=True)

        section_data['Option Type'] = "List"
        data = pd.concat([data, section_data], ignore_index=True)
        
        print('Section Data Size:', section_data.shape[0], '| Dedup Size:', section_data.drop_duplicates(['SKU','URL']).shape[0])
        print('All Data Size:', data.shape[0], '| Dedup Size:', data.drop_duplicates(['SKU','URL']).shape[0])
        continue

    else:
        pass
    

    try:
        page_data = get_page_data()
        page_data['Option Type'] = "Plain"
        page_data = cols_dedup(page_data)
        data = pd.concat([data, page_data], ignore_index=True)
        print('Page Data Size:', page_data.shape[0], '| All Data Size:',  data.shape[0], '| Dedup Size:', data.drop_duplicates(['SKU','URL']).shape[0])

    except:
        print('BAD LINK', url)
        continue


Link # 234 | Number of options: 29
Section Data Size: 0 | Dedup Size: 0
All Data Size: 2314 | Dedup Size: 255

Link # 235 | Number of options: 29
Section Data Size: 0 | Dedup Size: 0
All Data Size: 2314 | Dedup Size: 255

Link # 236 | Number of options: 33
Section Data Size: 0 | Dedup Size: 0
All Data Size: 2314 | Dedup Size: 255

Link # 237 | Number of options: 3
Section Data Size: 3 | Dedup Size: 3
All Data Size: 2317 | Dedup Size: 258

Link # 238 | Number of options: 3
Section Data Size: 0 | Dedup Size: 0
All Data Size: 2317 | Dedup Size: 258

Link # 239 | Number of options: 2
Section Data Size: 0 | Dedup Size: 0
All Data Size: 2317 | Dedup Size: 258

Link # 240 | Number of options: 5
Section Data Size: 0 | Dedup Size: 0
All Data Size: 2317 | Dedup Size: 258

Link # 241 | Number of options: 3
Section Data Size: 3 | Dedup Size: 3
All Data Size: 2320 | Dedup Size: 261

Link # 242 | Number of options: 3
Section Data Size: 0 | Dedup Size: 0
All Data Size: 2320 | Dedup Size: 261

Link #


Link # 315 | Number of options: 0
Page Data Size: 1 | All Data Size: 2370 | Dedup Size: 311

Link # 316 | Number of options: 0
Page Data Size: 1 | All Data Size: 2371 | Dedup Size: 312

Link # 317 | Number of options: 0
Page Data Size: 1 | All Data Size: 2372 | Dedup Size: 313

Link # 318 | Number of options: 0
Page Data Size: 1 | All Data Size: 2373 | Dedup Size: 314

Link # 319 | Number of options: 0
Page Data Size: 1 | All Data Size: 2374 | Dedup Size: 315

Link # 320 | Number of options: 0
Page Data Size: 1 | All Data Size: 2375 | Dedup Size: 316

Link # 321 | Number of options: 0
Page Data Size: 1 | All Data Size: 2376 | Dedup Size: 317

Link # 322 | Number of options: 0
Page Data Size: 1 | All Data Size: 2377 | Dedup Size: 318

Link # 323 | Number of options: 0
Page Data Size: 1 | All Data Size: 2378 | Dedup Size: 319

Link # 324 | Number of options: 0
Page Data Size: 1 | All Data Size: 2379 | Dedup Size: 320

Link # 325 | Number of options: 0
Page Data Size: 1 | All Data Size: 


Link # 398 | Number of options: 12
Section Data Size: 0 | Dedup Size: 0
All Data Size: 2456 | Dedup Size: 392

Link # 399 | Number of options: 12
Section Data Size: 0 | Dedup Size: 0
All Data Size: 2456 | Dedup Size: 392

Link # 400 | Number of options: 12
Section Data Size: 0 | Dedup Size: 0
All Data Size: 2456 | Dedup Size: 392

Link # 401 | Number of options: 0
Page Data Size: 1 | All Data Size: 2457 | Dedup Size: 393

Link # 402 | Number of options: 0
Page Data Size: 1 | All Data Size: 2458 | Dedup Size: 394

Link # 403 | Number of options: 0
Page Data Size: 1 | All Data Size: 2459 | Dedup Size: 395

Link # 404 | Number of options: 5
Section Data Size: 5 | Dedup Size: 5
All Data Size: 2464 | Dedup Size: 400

Link # 405 | Number of options: 5
Section Data Size: 0 | Dedup Size: 0
All Data Size: 2464 | Dedup Size: 400

Link # 406 | Number of options: 5
Section Data Size: 0 | Dedup Size: 0
All Data Size: 2464 | Dedup Size: 400

Link # 407 | Number of options: 5
Section Data Size: 0 | 

In [179]:
data.drop_duplicates(['SKU','URL']).shape, data.shape

((400, 86), (2464, 86))

In [292]:
data.to_csv('{}-RawData.csv'.format(website))

### Getting Data Functions

In [123]:
def get_page_data():
    # Title
    path = '//*[@class="product--title"]'
    title = browser.find_elements_by_xpath(path)[0].text
    
    # Sub Title
    try:
        path = '//*[@class="product--sub-title"]'
        sub_title = browser.find_elements_by_xpath(path)[0].text
    except:
        sub_title = np.nan
        
        
    # Short Descr
    try:
        path = '//*[@class="custom--bullet-points"]'
        short_desc = browser.find_elements_by_xpath(path)[0].text
        short_desc_df = pd.DataFrame([x.split(':') for x in short_desc.split('\n')])
        cols = []
        for idx in short_desc_df.index:
            if short_desc_df[1].loc[idx] != None:
                cols.append(short_desc_df[0].loc[idx])
            else:
                cols.append('Unknown') 

        short_desc_df[1] = short_desc_df[1].fillna(short_desc_df[0])
        short_desc_df = short_desc_df.T
        short_desc_df.columns = cols
        short_desc_df = short_desc_df.drop(0).reset_index(drop=True)
    except:
        short_desc_df = pd.DataFrame()
        
    
    # Item Price
    path = '//*[@class="price--content content--default"]'
    item_price = browser.find_elements_by_xpath(path)[0].text
    
    
    # Meta price
    
    try:
        path = '//*[@class="product--price price--unit"]' 
        price_meta_text = browser.find_elements_by_xpath(path)[0].text
    except:
        path = '//*[@class="product--tax"]'
        price_meta_text = browser.find_elements_by_xpath(path)[0].text
        
    path = '//*[@itemprop="price"]'
    price_meta = browser.find_elements_by_xpath(path)[0].get_attribute('content')
    
    
    # SKU
    path = '//*[@class="breadcrumb--entry is--active"]'
    sku = browser.find_elements_by_xpath(path)[0].text
    
    # Raw Description
    path = '//*[@class="custom--description"]'
    raw_desc = browser.find_elements_by_xpath(path)[0].text
    
    # Detailed Descr
    path = '//*[@class="custom--bulletpoints"]'
    bullets = browser.find_elements_by_xpath(path)[0].text
    bullets_df = pd.DataFrame([x.split(':') for x in bullets.split('\n')])
    cols = []
    for idx in bullets_df.index:
        if bullets_df[1].loc[idx] != None:
            cols.append(bullets_df[0].loc[idx])
        else:
            cols.append('Unknown') 

    bullets_df[1] = bullets_df[1].fillna(bullets_df[0])
    bullets_df = bullets_df.T
    bullets_df.columns = cols
    bullets_df = bullets_df.drop(0).reset_index(drop=True)

    # Main Data
    page_df = pd.concat([short_desc_df, bullets_df], axis=1)
    page_df = cols_dedup(page_df)
    
    page_df['Title'] = title
    page_df['SubTitle'] = sub_title
    page_df['Price Item'] = item_price
    page_df['Price Meta Text'] = price_meta_text
    page_df['Price Meta'] = price_meta
    page_df['SKU'] = sku
    page_df['URL'] = browser.current_url
    
    return page_df

In [125]:
#Check Dupl Columns
def cols_dedup(data):
    cols = []
    dupl_cols = []
    dupl_idx = []

    for i, col in enumerate(data.columns):
        if col not in cols:
            cols.append(col)
        else:
            dupl_cols.append(col)
            dupl_idx.append(i)


    if dupl_cols:
        new_cols = data.columns.to_list()
        count=0
        for idx in dupl_idx:
            count+=1
            new_cols[idx] = new_cols[idx]+'-'+str(count)
        data.columns = new_cols
    return data

# Data Processing

In [4]:
data = pd.read_csv('{}-RawData.csv'.format(website), index_col=[0])

In [5]:
df_proc = data.copy().reset_index(drop=True)
df_proc.shape

(2464, 86)

In [6]:
df_proc = df_proc.drop_duplicates(['SKU', 'URL', 'Price Item'])
df_proc.shape

(400, 86)

### Combine all columns to mine text form

In [7]:
df_proc['Combined'] = ''
for col in df_proc.columns:
    df_proc['Combined'] = df_proc['Combined'] + '; ' + df_proc[col].astype(str)

In [8]:
df_proc['Combined'] = df_proc['Combined'].str.replace(r'\; nan','').str.replace(r'^; ','')

### Keywords

In [253]:
from df2gspread import gspread2df as g2d
from oauth2client.service_account import ServiceAccountCredentials
spreadsheet_key = '1iDGux_PxTPSIkxnljeEhMifJo5jAXDhxOXLB8zlrOKg'
scope = ['https://spreadsheets.google.com/feeds'] 
credentials = ServiceAccountCredentials.from_json_keyfile_name('/Users/macbook/Desktop/UpWork/gs-credentials.json', scope) 
keywords = g2d.download(gfile=spreadsheet_key, wks_name = 'Keywords', credentials=credentials, col_names=True, row_names=True)

In [254]:
rename_cols = {'Produktart':'Product Type', 
               'Holztyp':'Wood Type',
               'Qualität':'Quality',
               'Oberflächenqualität':'Surface Quality',
               'Sortierklasse':'Sorting Class',
               'Trocknung':'Drying Method'}
keywords = keywords.rename(columns=rename_cols)

In [255]:
for col in keywords.columns:
    keywords[col] = keywords[col].apply(lambda x: np.nan if x == '' or x == ' ' or x== '-777' else x)

###  Quality

In [345]:
df_proc['Comb Quality'] = ''
for i in df_proc.index:
    df_proc['Comb Quality'].loc[i] = []
    
    for qual in keywords['Quality'].unique():
        qual = str(qual) 
        pat = r' {} |^{} | {}$| {}\;'.format(qual.lower(), qual.lower(), qual.lower(), qual.lower())
        
        if qual!= 'nan' and len(re.findall(pat, df_proc['Combined'].loc[i].lower()))>0 and qual.lower() not in ', '.join(df_proc['Quality'].loc[i]):
            df_proc['Comb Quality'].loc[i].append(qual)
            
    try:
        df_proc['Comb Quality'].loc[i] = ', '.join(df_proc['Comb Quality'].loc[i])
    except:
        pass
df_proc['Comb Quality'] =  df_proc['Comb Quality'].apply(lambda x: np.nan if x==' ' or x=='' else x )

df_proc['Quality'] = df_proc['Sortierung'].fillna(df_proc['Comb Quality'])
df_proc['Quality'] = df_proc['Quality'].str.replace(r'^ ', '')



df_proc['Quality'].value_counts()

1. Wahl                 47
B/B                     18
Sortierklasse A/B       17
A/B-Sortierung          16
B-Sortierung            11
2. Wahl (Naturwuchs)    10
B/C                      8
A-Sortierung             7
Altholz                  5
Sortierklasse C          3
Hobelfallend             1
A/B                      1
Name: Quality, dtype: int64

### Sorting Class

In [344]:
df_proc['Sorting Class'] = ''
for i in df_proc.index:
    df_proc['Sorting Class'].loc[i] = []
    
    for sort_class in keywords['Sorting Class'].unique():
        sort_class = str(sort_class)
        pat = r' {} |^{} | {}$| {}\;'.format(sort_class.lower(), sort_class.lower(), sort_class.lower(), sort_class.lower())
        if len(re.findall(pat, df_proc['Combined'].loc[i].lower()))>0 and sort_class.lower() not in ', '.join(df_proc['Sorting Class'].loc[i]):
            df_proc['Sorting Class'].loc[i].append(sort_class)
       
    try:
        df_proc['Sorting Class'].loc[i] = ', '.join(df_proc['Sorting Class'].loc[i])
    except:
        pass
df_proc['Sorting Class'] =  df_proc['Sorting Class'].apply(lambda x: np.nan if x==' ' or x=='' else x )
df_proc['Sorting Class'].value_counts()

Series([], Name: Sorting Class, dtype: int64)

## Main parameters

### Wood Types

In [310]:
df_proc['Meta Wood'] = ''
for i in df_proc.index:
    for wood in keywords['Wood Type'].unique():
        wood = str(wood)
        pat = r' {} |^{} | {}$'.format(wood.lower(), wood.lower(), wood.lower())
        if len(re.findall(pat, df_proc['Combined'].loc[i].lower()))>0:
            df_proc['Meta Wood'].loc[i] = wood

In [311]:
df_proc['Meta Wood'] = df_proc['Meta Wood'].apply(lambda x: np.nan if x==' ' or x=='' else x )

In [312]:
df_proc['Wood Type'] = df_proc['Meta Wood']
for col in ['Material', 'Material-2', 'Material-4','Holzart-5','Holzart-3','Holzart-4','Holzart']:
    df_proc['Wood Type'] = df_proc['Wood Type'].fillna(df_proc[col])

In [313]:
df_proc['Wood Type'] = df_proc['Wood Type'].apply(lambda x: np.nan if x==' ' or x=='' else x )

In [314]:
df_proc['Wood Type'].isna().sum()

15

#### Cheking wrong typs and woods

In [323]:
wrong_wood = ['Holzwerkstoff (MDF)', 'Glas', 'Aluminium', 'Metall', 'Granit', 'Polymere', 'Granito',
              'Kunststoff','Laminiert','Gipskarton, Furniert','PVC','Beschichtet', 'Melaminbeschichtet' ,
              'Furniert','PVC-freie Werkstoffmischung',  'Foliert',  'Laminiert', 'Polymere','Polycarbonat',
              'Polyvinylchlorid (PVC)','Gipskarton','German Compact Composite (GCC)','Bandstahl','Foliert',
              'Faseprofil','Metall, Kunststoff', 'Textil, Kunststoff','Rotbuche','Multiplex',
              'Qualitätsspanplatte mit HPL-Beschichtung Profil aus Hart-PVC','Holz-Kunststoff-Verbundwerkstoffe (WPC)'
              ,'Beschichtete Flachpressplatte', 'Hart-PVC-Leiste','Lackiert','WPC', 
              'Weich-PVC','Unbehandelt','Kunststoffbeschichtet','Kunststoffummantelter Hartfaserkern',
               'Stahl','Beschichtete Flachpressplatte, Hart-PVC-Leiste','Beidseitig mit HPL beschichtete Spanplatte',
              'Polycarbonat', 'German Compact Composite (GCC)', 
              'Foliert', 'WPC', 'Multiplex', 'Beschichtet', 'Stahl','Metall, Verzinkt','Kunststoff, Metall',
             'Verbundwerkstoff','Beton','Edelstahl','Normalbeton','Chrom-Vanadium-Stahl','Holz, Stahl',
             'Stahl verzinkt','Gummigranulat','Hartkunststoff','Kunststoff, Stahl','Aluminium, Holz',
             'Polyethylen','Holz, Kunststoff','Polypropylen','Moltopren-Schaumstoff','Wasserbasiert',
             'Holzfaser','PVC-frei','Stahl, Kunststoff','Gummi']

In [324]:
# Percentage of wrong wood types
df_proc[(df_proc['Wood Type'].isin(wrong_wood))].shape[0]/df_proc.shape[0]*100

0.5

In [325]:
df_proc[(df_proc['Wood Type'].isin(wrong_wood))]['Wood Type'].value_counts()

Gummi                1
Stahl, Kunststoff    1
Name: Wood Type, dtype: int64

In [326]:
df_proc.shape

(400, 104)

In [327]:
df_proc = df_proc[(~df_proc['Wood Type'].isin(wrong_wood))]
df_proc.shape

(398, 104)

In [328]:
df_proc['Wood Type'] = df_proc['Wood Type'].str.replace(r'^ ','')

In [329]:
df_proc['Wood Type'].value_counts()

Fichte/Tanne           120
Kiefer                  88
Fichte                  36
Douglasie               31
Nadelholz               27
Buche                   22
Eiche                   20
Weisstanne               7
Bangkirai                6
Fichte/Tanne/Kiefer      5
Akazie                   4
Lärche                   3
Sibirische Lärche        3
Mandioqueira             3
Thermoesche              2
sib. Lärche              1
Ahorn                    1
Garapa                   1
Holz                     1
Fchte                    1
Walnuss                  1
Name: Wood Type, dtype: int64

### Surface Treatment

In [322]:
strong_surfaces = ['geschliffen', 'gehobelt', 'glatte Oberfläche', 'gefast',
                   'sägerau', 'geriffelt', 'glatt gehobelt', 'glatte', 'gefräst',
                   'gebürstet', 'strukturiert', '1-Seitig gehobelt',
                   '2-Seitig gehobelt', '3-Seitig gehobelt', '4-Seitig gehobelt',
                   'genutet', 'unbehandelt', 'naturbelassen', 'deckend',
                   'lasiert', 'vorgeölt', 'kesseldruckimprägniert (KDI)',
                   'hitzebehandelt', 'thermobehandelt', 'wachsen', 'imprägniert',
                   'Scharfkantig', 'gespitzt', 'N+F Stirnseite',
                   'abgerundet', 'Nut und Feder']

weak_surfaces = ['roh', 'keine']

In [305]:
df_proc['Combined Surface Treat'] = ''
for i in df_proc.index:
    df_proc['Combined Surface Treat'].loc[i] = []
    for surface in weak_surfaces:
        pat = r' {} |^{} | {}$'.format(surface.lower(), surface.lower(), surface.lower())
        if len(re.findall(pat, df_proc['Combined'].loc[i].lower()))>0 and surface.lower() not in ', '.join(df_proc['Combined Surface Treat'].loc[i]):
            df_proc['Combined Surface Treat'].loc[i].append(surface)
    
    for surface in strong_surfaces:
        if surface.lower() in df_proc['Combined'].loc[i].lower() and surface.lower() not in ', '.join(df_proc['Combined Surface Treat'].loc[i]):
            df_proc['Combined Surface Treat'].loc[i].append(surface)
    try:
        df_proc['Combined Surface Treat'].loc[i] = ', '.join(df_proc['Combined Surface Treat'].loc[i])
    except:
        pass

In [306]:
df_proc['Combined Surface Treat'] = df_proc['Combined Surface Treat'].apply(lambda x: np.nan if x==' ' or x=='' else x )

In [307]:
df_proc['Surface Treatment'] = df_proc['Oberfläche'] + ',' + df_proc['Oberflächenbehandlung']
for col in ['Oberfläche-4','Combined Surface Treat']:
    df_proc['Surface Treatment'] = df_proc['Surface Treatment'].fillna(df_proc[col]).str.lower()

In [290]:
df_proc['Surface Treatmeant'].value_counts()

gehobelt, gefast, 4-seitig gehobelt                 60
gehobelt                                            59
geschliffen, gefast                                 57
sägerau                                             32
imprägniert                                         31
geschliffen                                         30
geriffelt, unbehandelt                              14
gehobelt, 4-seitig gehobelt                         10
gehobelt, gefast                                     8
unbehandelt                                          6
gehobelt, gefast, 4-seitig gehobelt, unbehandelt     6
genutet                                              6
imprägniert, gespitzt                                5
naturbelassen                                        5
geriffelt, imprägniert                               5
gehobelt, gefast, imprägniert                        5
sägerau, imprägniert                                 4
geschliffen, unbehandelt                             4
gebürstet 

### Surface Quality

In [32]:
df_proc['Surface Quality'] = ''
for i in df_proc.index:
    df_proc['Surface Quality'].loc[i] = []
    
    for surf_qual in keywords['Surface Quality'].dropna().unique():
        pat = r' {} |^{} | {}$|[a-z] {}\;'.format(surf_qual.lower(), surf_qual.lower(), surf_qual.lower(), surf_qual.lower())
        if len(re.findall(pat, df_proc['Combined'].loc[i].lower()))>0 and surf_qual.lower() not in ', '.join(df_proc['Surface Quality'].loc[i]):
            df_proc['Surface Quality'].loc[i].append(surf_qual)
            
    try:
        df_proc['Surface Quality'].loc[i] = ', '.join(df_proc['Surface Quality'].loc[i])
    except:
        pass
df_proc['Surface Quality'] =  df_proc['Surface Quality'].apply(lambda x: np.nan if x==' ' or x=='' else x)

In [33]:
df_proc['Surface Quality'].value_counts()

NSI    19
Name: Surface Quality, dtype: int64

### Drying method

In [34]:
df_proc['Combined Drying Method'] = ''
for i in df_proc.index:
    df_proc['Combined Drying Method'].loc[i] = []
    for method in keywords['Drying Method'].dropna().unique():
        method = str(method) 
        
        pat = r' {} |^{} | {}$| {}\;'.format(method.lower(), method.lower(), method.lower(), method.lower())
        if method!='nan' and len(re.findall(pat, df_proc['Combined'].loc[i].lower()))>0 and method.lower() not in ', '.join(df_proc['Combined Drying Method'].loc[i]):
            df_proc['Combined Drying Method'].loc[i].append(method)

    try:
        df_proc['Combined Drying Method'].loc[i] = ', '.join(df_proc['Combined Drying Method'].loc[i])
    except:
        pass

In [35]:
df_proc['Combined Drying Method'] =  df_proc['Combined Drying Method'].apply(lambda x: np.nan if x==' ' or x=='' else x )

In [36]:
df_proc['Drying Method'] = df_proc['Holzfeuchte'].fillna(df_proc['Combined Drying Method']).fillna(df_proc['Holzfeuchte-3'])

In [37]:
df_proc['Drying Method'].value_counts()

technisch getrocknet    12
 12% (+/- 2%)            5
Name: Drying Method, dtype: int64

## Dimentions

In [38]:
df_proc.head(1)

,Länge,Breite,Stärke,Farbe,Länge-1,Breite-2,Stärke-3,Farbe-4,Unknown,Unknown-5,Unknown-6,Title,SubTitle,Price Item,Price Meta Text,Price Meta,SKU,URL,Option Type,Holzart,Holzart-4,Oberflächenbehandlung,Oberfläche,Höhe,Farbe-2,Höhe-3,Unknown-7,Einsatz,Material,Setz bestehend aus,Einsatz-1,Material-2,Unknown-3,Setz bestehend aus-4,Unknown-1,Inhalt,Unknown-4,Inhalt-5,Unknown-2,schnelle Montage,Material-4,Lieferumfang,Anwendungsbereich,Dauerhaftigkeit nach DIN Dauerhaftigkeitsklasse,zertifiziert nach,Dauerhaftigskeitsklasse,Dauerhaftigkeitsklasse,Hinweis,Stärke-2,Holzart-3,Sortierung,Packinhalt,Deckfläche,Profilart,Holzart-5,Paketinhalt,Unknown-8,Breite-1,Oberfläche-4,Länge-3,Verlegung,Unknown-9,Maße,Holzfeuchte,Maße-1,Holzfeuchte-3,Vielfälltige Einsatzmöglichkeiten,Kanten,Profil,Dekor,Dekor-4,Optik,Nutzschichtdicke,Fugenbild,Veredelung,Aufbau,Mittellage,Länge-2,Optil,Trägerplatt,Nutzungsklasse,Klick-Verbindung,Maße-3,Größe,Größe-2,Reichweite pro Platte,Combined,Quality,Sorting Class,Meta Wood,Wood Type,Combined Surface Treat,Surface Treatment,Surface Quality,Combined Drying Method,Drying Method
0,300 mm,300 mm,21 mm,grau,300 mm,300 mm,21 mm,grau,leichte Montage durch Steck-Verbindungssystem,zur Verlegung auf festem Untergrund im Innen- ...,"10 Stück im Karton = 0,90 m²",TrendLine WPC Fliese grau,"300 x 300 mm (L x B), Stärke: 21 mm","38,98 € / m²","Inhalt: 0,9 m² Stückpreis: 35,08 €)","35,08",780800368,https://www.globus-baumarkt.de/trendline-wpc-f...,Plain,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,300 mm; 300 mm; 21 mm; grau; 300 mm; 300...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [123]:
for col in ['Länge-1', 'Länge-3', 'Länge-2']:
    df_proc['Länge'] = df_proc['Länge'].fillna(df_proc[col])
for col in ['Breite-2', 'Breite-1']:
    df_proc['Breite'] = df_proc['Breite'].fillna(df_proc[col])
for col in ['Stärke-3', 'Stärke-2']:
    df_proc['Stärke'] = df_proc['Stärke'].fillna(df_proc[col])

In [124]:
df_proc[['Länge','Stärke','Breite']].isna().sum()/df_proc.shape[0]*100

Länge      4.75
Stärke     4.00
Breite    13.25
dtype: float64

In [125]:
df_proc['SubTitle'].str.split(' x ', expand=True).isna().sum()/df_proc.shape[0]

0    0.00
1    0.06
2    0.10
dtype: float64

In [126]:
title_split = df_proc['SubTitle'].str.split(' x ', expand=True) #.str.replace(r'^.+?(?=\d)','')
pat = r'((?<=\d| )cm)|((?<=\d| )mm)|((?<=\d| )m(?!=m))'
mult_df = pd.DataFrame([])

for col in range(3):
    temp_split = title_split[col].str.extract(pat, expand=True)
    for col in temp_split.columns[1:]:
        temp_split[0] = temp_split[0].fillna(temp_split[col])
    
    mult_df = pd.concat([mult_df, temp_split[[0]]], axis=1)

In [127]:
mult_df.columns = ['Firts Mult', 'Second Mult', 'Third Mult']
mult_df['Firts Mult'] = mult_df['Firts Mult'].fillna(mult_df['Third Mult']).fillna(mult_df['Second Mult'])
mult_df['Second Mult'] = mult_df['Second Mult'].fillna(mult_df['Third Mult'])

In [128]:
mult_df = pd.concat([mult_df, df_proc[['SubTitle','Länge','Stärke','Breite']]], axis=1)

In [129]:
mult_df['SubTitle Num'] = mult_df['SubTitle'].str.replace(r'^\D+?(?=\d)','')
mult_df['SubTitle Num'] = mult_df['SubTitle Num'].str.replace(r'(?<=^\d)\.(?=\d\d)','')
mult_df['SubTitle Num'] = mult_df['SubTitle Num'].str.replace(r'(?<=\d)\,(?=\d)','.')
mult_df['SubTitle Num'] = mult_df['SubTitle Num'].str.replace(r'(?<=\d)[^0-9\.]+(?=\d)','x')
mult_df['SubTitle Num'] = mult_df['SubTitle Num'].str.replace(r'[^0-9\.x]','')

In [131]:
mult_df['Firts Mult Num'] = mult_df['Firts Mult'].fillna('-999').str.replace('mm','1').str.replace('cm','10').str.replace('m','1000').astype(float)
mult_df['Second Mult Num'] = mult_df['Second Mult'].fillna('-999').str.replace('mm','1').str.replace('cm','10').str.replace('m','1000').astype(float)
mult_df['Third Mult Num'] = mult_df['Third Mult'].fillna('-999').str.replace('mm','1').str.replace('cm','10').str.replace('m','1000').astype(float)

In [132]:
pat = r'((?<=\d| )cm)|((?<=\d| )mm)|((?<=\d| )m(?!=m))'
temp_split = mult_df['Länge'].str.extract(pat, expand=True)
for col in temp_split.columns[1:]:
        temp_split[0] = temp_split[0].fillna(temp_split[col])
mult_df['Länge Mult'] = temp_split[0]

In [133]:
temp_split = mult_df['Stärke'].str.extract(pat, expand=True)
for col in temp_split.columns[1:]:
        temp_split[0] = temp_split[0].fillna(temp_split[col])
mult_df['Stärke Mult'] = temp_split[0]

In [134]:
temp_split = mult_df['Breite'].str.extract(pat, expand=True)
for col in temp_split.columns[1:]:
        temp_split[0] = temp_split[0].fillna(temp_split[col])
mult_df['Breite Mult'] = temp_split[0]

In [135]:
for col in ['Länge Mult', 'Breite Mult', 'Stärke Mult']:
    mult_df[col] = mult_df[col].fillna('-99999').astype(str).str.replace('mm','1').str.replace('cm','10').str.replace('m','1000').astype(float)


In [137]:
for col in ['Länge','Stärke','Breite']:
    print(col)
    mult_df[col + ' Num'] = mult_df[col].str.replace(r'^\D+?(?=\d)','')
    mult_df[col + ' Num'] = mult_df[col + ' Num'].str.replace(r'(?<=^\d)\.(?=\d\d)','')
    mult_df[col + ' Num'] = mult_df[col + ' Num'].str.replace(r'(?<=\d)\,(?=\d)','.')
    mult_df[col + ' Num'] = mult_df[col + ' Num'].str.replace(r'(?<=\d)[^0-9\.]+(?=\d)','x')
    mult_df[col + ' Num'] = mult_df[col + ' Num'].str.replace(r'[^0-9\.]','')
    mult_df[col + ' Num'] = mult_df[col + ' Num'].astype(float)

Länge
Stärke
Breite


In [138]:
mult_cols = ['Länge Mult', 'Breite Mult', 'Stärke Mult']
for i, col in enumerate(['Länge Num', 'Breite Num', 'Stärke Num']):
    mult_df[col] = mult_df[col]*mult_df[mult_cols[i]]

In [139]:
mult_df['Title Dims Num'] = np.nan
for idx in mult_df.index:
    
    mults = mult_df.loc[idx, 'Firts Mult Num':'Third Mult Num'].values
    
    raw_dims = []
    for i, dim in enumerate(mult_df['SubTitle Num'].loc[idx].split('x')):
        try:
            dim = float(dim)*mults[i]
            if dim>0:
                raw_dims.append(dim)
        except:
            pass
    mult_df['Title Dims Num'].loc[idx] = 'x'.join([str(x) for x in raw_dims])

In [140]:
import math
for idx in mult_df.index:
    title_dims = mult_df['Title Dims Num'].loc[idx].split('x')
    starke = mult_df['Stärke Num'].loc[idx]
    lange = mult_df['Länge Num'].loc[idx]
    breite = mult_df['Breite Num'].loc[idx]
    
    if math.isnan(starke) and str(lange) in title_dims and str(breite) in title_dims:
        try:
            mult_df['Stärke Num'].loc[idx] = [x for x in title_dims if x!=str(lange) and x !=str(breite)][0]
        except:
            pass

In [141]:
mult_df[['Länge Num', 'Breite Num', 'Stärke Num']].isna().sum()/mult_df.shape[0]*100

Länge Num      4.75
Breite Num    13.25
Stärke Num     4.00
dtype: float64

In [142]:
for idx in mult_df[mult_df['Stärke'].fillna("-").str.contains('x')].index:
    mult_df['Stärke Num'].loc[idx] = np.nan
    mult_df['Breite Num'].loc[idx] = np.nan

In [143]:
for idx in mult_df.index:
    try:
        dims = [float(x) for x in mult_df['Title Dims Num'].loc[idx].split("x")]
    except:
        continue
        
    # L
    try:
        if len(dims)==3 and math.isnan(mult_df['Länge Num'].loc[idx]):
            mult_df['Länge Num'].loc[idx] = np.max(dims)
    except:
        pass
    
    # S
    try:
        if len(dims)==3 and math.isnan(mult_df['Stärke Num'].loc[idx]):
            mult_df['Stärke Num'].loc[idx] = np.min(dims)
    except:
        pass
    
    # B
    try:
        if len(dims)==3 and math.isnan(mult_df['Breite Num'].loc[idx]):
            mult_df['Breite Num'].loc[idx] = np.median(dims)
    except:
        pass

In [144]:
mult_df[['Länge Num', 'Breite Num', 'Stärke Num']].isna().sum()/mult_df.shape[0]*100

Länge Num     2.5
Breite Num    1.5
Stärke Num    2.5
dtype: float64

In [147]:
mult_df.sample(5)

,Firts Mult,Second Mult,Third Mult,SubTitle,Länge,Stärke,Breite,SubTitle Num,Firts Mult Num,Second Mult Num,Third Mult Num,Länge Mult,Stärke Mult,Breite Mult,Länge Num,Stärke Num,Breite Num,Title Dims Num
2430,mm,mm,mm,"27 x 144 x 2500 mm, getrocknet, gehobelt, Kant...",2500mm,27 mm,144 mm,27x144x2500,1.0,1.0,1.0,1.0,1.0,1.0,2500.0,27.0,144.0,27.0x144.0x2500.0
346,mm,mm,mm,3000 x 121 x 14 mm,3000 mm,14 mm,121 mm,3000x121x14,1.0,1.0,1.0,1.0,1.0,1.0,3000.0,14.0,121.0,3000.0x121.0x14.0
2441,NaN,NaN,NaN,Eiche rustikal geölt weiß,2200 mm,"11,5 mm",185 mm,,-999.0,-999.0,-999.0,1.0,1.0,1.0,2200.0,11.5,185.0,
2431,mm,mm,mm,"27 x 144 x 2500 mm, getrocknet, gehobelt, Kant...",2500 mm,27 mm,144 mm,27x144x2500,1.0,1.0,1.0,1.0,1.0,1.0,2500.0,27.0,144.0,27.0x144.0x2500.0
68,mm,mm,mm,"3000 x 70 x 45 mm, unbehandelt",3000 mm,45 mm,70 mm,3000x70x45,1.0,1.0,1.0,1.0,1.0,1.0,3000.0,45.0,70.0,3000.0x70.0x45.0


In [148]:
df_proc = pd.concat([df_proc, mult_df[['Länge Num', 'Stärke Num','Breite Num']]], axis=1)

In [149]:
df_proc = df_proc.rename(columns = {'Länge Num':'Lange mm', 'Stärke Num':'Stärke mm', 'Breite Num':'Breite mm'})

### Price

In [171]:
df_proc['Price Num'] = df_proc['Price Item']
for idx in df_proc[df_proc['Price Item'].str.contains('\/')].index:
    df_proc['Price Num'].loc[idx] = df_proc['Price Meta'].loc[idx]

In [174]:
price_cols = ['Price Item','Price Meta Text','Price Meta','Price Num']
df_proc[df_proc['Price Item'].str.contains('\/')][price_cols].sample(2)

,Price Item,Price Meta Text,Price Meta,Price Num
2440,"23,88 € / m²","Inhalt: 2,198 m² Paketpreis: 52,49 €)","52,49","52,49"
211,"0,78 € / lfm","Inhalt: 3 lfm Stückpreis: 2,33 €)","2,33","2,33"


In [182]:
df_proc['Price Num'] = df_proc['Price Num'].str.replace('[^\d\,\.]','')
df_proc['Price Num'] = df_proc['Price Num'].str.replace('\,','.')
df_proc['Price Num'] = df_proc['Price Num'].astype(float)

### SKU

In [190]:
df_proc['SKU'] = df_proc['SKU'].str.replace(r'[^\d]','')

### Images

In [192]:
browser =  webdriver.Chrome('/Users/macbook/Downloads/Chrome Driver/chromedriver', chrome_options=chrome_options)

In [191]:
images_df = pd.DataFrame({'Images':np.nan, 'URL':np.nan}, index=[df_proc.index])
images_df.shape

(400, 2)

In [242]:
start=10
#end=60
for p, idx in enumerate(images_df.index[start:], start=start):
    
    print('\nProduct', p)
    
    
    url = df_proc['URL'].loc[idx]
    browser.get(url)
    sleep(1)
    
    img_urls = []
    path = '//*[@class="image--box image-slider--item"]/span'
    img_urls_raw = browser.find_elements_by_xpath(path)
    for i in range(len(img_urls_raw)):
        img_url = img_urls_raw[i].get_attribute('data-img-large')
        print(img_url)
        img_urls.append(img_url)

    
    if p%100==0:
        images_df.to_csv('Images-{}'.format(website))
    
    images_df['URL'].loc[idx]=url
    images_df['Images'].loc[idx]= '; '.join(img_urls)


Product 10
https://www.globus-baumarkt.de/media/image/a4/e9/ca/4006850337163_1280x1280.jpg

Product 11
https://www.globus-baumarkt.de/media/image/11/7f/4a/4012271069651_1280x1280.jpg

Product 12
https://www.globus-baumarkt.de/media/image/f6/f3/cc/4002718269454_1280x1280.jpg

Product 13
https://www.globus-baumarkt.de/media/image/15/4a/7e/4002718641731_1280x1280.jpg

Product 14
https://www.globus-baumarkt.de/media/image/40/ec/1d/4005014409005_1280x1280.jpg

Product 15
https://www.globus-baumarkt.de/media/image/a0/53/13/4005014572211_1280x1280.jpg

Product 16
https://www.globus-baumarkt.de/media/image/02/6b/a6/4005014481582_1280x1280.jpg

Product 17
https://www.globus-baumarkt.de/media/image/85/b1/8f/4005014661632_Terrassendiele_mitNut_KI_thermo_Produktbild_1280x1280.jpg

Product 18
https://www.globus-baumarkt.de/media/image/0f/35/7d/4005014661649_1280x1280.jpg

Product 19
https://www.globus-baumarkt.de/media/image/7e/47/60/4005014661663_Terrassendiele_DG_select-20mit-20Nut_Produktbildrx

https://www.globus-baumarkt.de/media/image/3a/71/01/4005014019228_1280x1280.jpg

Product 66
https://www.globus-baumarkt.de/media/image/25/c8/b6/4005014019372_1280x1280.jpg

Product 67
https://www.globus-baumarkt.de/media/image/b6/60/eb/4005014019259_1280x1280.jpg

Product 68
https://www.globus-baumarkt.de/media/image/65/53/b1/4005014019365_1280x1280.jpg

Product 69
https://www.globus-baumarkt.de/media/image/d8/fc/7d/4005014019341_1280x1280.jpg

Product 70
https://www.globus-baumarkt.de/media/image/2a/f9/49/4005014490065_1280x1280.jpg

Product 71
https://www.globus-baumarkt.de/media/image/f8/8d/46/4005014019303_1280x1280.jpg

Product 72
https://www.globus-baumarkt.de/media/image/f4/98/72/4005014019334_1280x1280.jpg

Product 73
https://www.globus-baumarkt.de/media/image/e8/ed/6b/4005014019327_1280x1280.jpg

Product 74
https://www.globus-baumarkt.de/media/image/43/db/88/2077851537175_1280x1280.jpg

Product 75
https://www.globus-baumarkt.de/media/image/43/db/88/2077851537175_1280x1280.jpg


https://www.globus-baumarkt.de/media/image/b6/6b/f7/5703393726070_1280x1280.jpg
https://www.globus-baumarkt.de/media/image/34/a9/bc/5703393726070_2020600_1280x1280.jpg

Product 99
https://www.globus-baumarkt.de/media/image/7f/19/be/5703393714596_1_1280x1280.jpg
https://www.globus-baumarkt.de/media/image/a1/f8/51/5703393714596_2_1280x1280.jpg
https://www.globus-baumarkt.de/media/image/b7/94/7f/5703393714596_1978290_1280x1280.jpg
https://www.globus-baumarkt.de/media/image/8f/28/89/5703393714596_1280x1280.jpg

Product 100
https://www.globus-baumarkt.de/media/image/7d/14/8c/5703393714602_1280x1280.jpg
https://www.globus-baumarkt.de/media/image/ba/a4/a5/5703393714602_1978291_1280x1280.jpg

Product 101
https://www.globus-baumarkt.de/media/image/25/e0/7a/5703393181435_1-202wQuUAkSqumxZz_1280x1280.jpg
https://www.globus-baumarkt.de/media/image/d9/81/9e/5703393181435_1_1280x1280.jpg
https://www.globus-baumarkt.de/media/image/44/0c/5b/5703393181435_2_1280x1280.jpg
https://www.globus-baumarkt.de/

https://www.globus-baumarkt.de/media/image/48/b3/fe/4005014014384_1280x1280.jpg

Product 159
https://www.globus-baumarkt.de/media/image/48/79/g0/4005014014377_1280x1280.jpg

Product 160
https://www.globus-baumarkt.de/media/image/bf/bd/d6/4005014000134_1280x1280.jpg

Product 161
https://www.globus-baumarkt.de/media/image/10/4d/a7/4005014000110_1280x1280.jpg

Product 162
https://www.globus-baumarkt.de/media/image/7d/dd/68/4005014000103_1280x1280.jpg

Product 163
https://www.globus-baumarkt.de/media/image/7b/9b/e4/4005014578381_1280x1280.jpg

Product 164
https://www.globus-baumarkt.de/media/image/21/70/62/4005014558932_1280x1280.jpg

Product 165
https://www.globus-baumarkt.de/media/image/0d/a9/50/4005014014469_1280x1280.jpg

Product 166
https://www.globus-baumarkt.de/media/image/a8/fc/6b/4005014014452_1280x1280.jpg

Product 167
https://www.globus-baumarkt.de/media/image/42/20/71/4005014014445_1280x1280.jpg

Product 168
https://www.globus-baumarkt.de/media/image/2f/dc/ac/4005014014438_1280

https://www.globus-baumarkt.de/media/image/e9/be/e1/4005014000936_1280x1280.jpg

Product 247
https://www.globus-baumarkt.de/media/image/68/82/d8/4005014009427_1280x1280.jpg

Product 248
https://www.globus-baumarkt.de/media/image/49/df/d3/4005014430641_1280x1280.jpg

Product 249
https://www.globus-baumarkt.de/media/image/d9/c7/72/4005014430665_1280x1280.jpg

Product 250
https://www.globus-baumarkt.de/media/image/60/89/af/4005014427863_1280x1280.jpg

Product 251
https://www.globus-baumarkt.de/media/image/91/c9/41/4005014427870_1280x1280.jpg

Product 252
https://www.globus-baumarkt.de/media/image/91/d5/a5/4005014322434_1280x1280.jpg

Product 253
https://www.globus-baumarkt.de/media/image/fc/0f/f1/4005014006044_1280x1280.jpg

Product 254
https://www.globus-baumarkt.de/media/image/2c/eb/dd/4005014006068_1280x1280.jpg

Product 255
https://www.globus-baumarkt.de/media/image/82/4f/6e/4005014498177_1280x1280.jpg

Product 256
https://www.globus-baumarkt.de/media/image/44/d4/31/4005014498191_1280

https://www.globus-baumarkt.de/media/image/07/85/12/4005014403126_1280x1280.jpg

Product 336
https://www.globus-baumarkt.de/media/image/0d/0f/0c/4005014403027_1280x1280.jpg

Product 337
https://www.globus-baumarkt.de/media/image/3f/25/b6/4005014015640_1280x1280.jpg

Product 338
https://www.globus-baumarkt.de/media/image/78/51/ed/4005014559137_1280x1280.jpg

Product 339
https://www.globus-baumarkt.de/media/image/6a/16/ac/4005014661847_1280x1280.jpg

Product 340
https://www.globus-baumarkt.de/media/image/3a/f5/27/k640_051589_rhombusprofil_dg_produktbild_-002V0h4J5HE3fSDL_1280x1280.jpg
https://www.globus-baumarkt.de/media/image/54/89/30/k640_051589_rhombusprofil_dg_produktbild_-002sh4ShMOd8QVJ2_1280x1280.jpg
https://www.globus-baumarkt.de/media/image/3e/96/8f/k640_051589_rhombusprofil_dg_produktbild_-002h2SVNmvOknOHB_1280x1280.jpg
https://www.globus-baumarkt.de/media/image/7e/22/31/k640_051589_rhombusprofil_dg_produktbild_-002gM2vulZgALve1_1280x1280.jpg
https://www.globus-baumarkt.de/medi

https://www.globus-baumarkt.de/media/image/a9/9c/7b/4014809124832_produkt_1280x1280.jpg
https://www.globus-baumarkt.de/media/image/18/85/e2/4014809124832_produkt1_1280x1280.jpg
https://www.globus-baumarkt.de/media/image/71/1a/53/4014809124832_produkt2_1280x1280.jpg
https://www.globus-baumarkt.de/media/image/b8/2b/eb/4014809124832_produkt3_1280x1280.jpg

Product 386
https://www.globus-baumarkt.de/media/image/64/c0/68/4014809124870_produkt1_1280x1280.jpg
https://www.globus-baumarkt.de/media/image/d1/6c/21/4014809124870_produkt_1280x1280.jpg
https://www.globus-baumarkt.de/media/image/cd/18/48/4014809124870_produkt2_1280x1280.jpg
https://www.globus-baumarkt.de/media/image/8e/e7/73/4014809124870_produkt3_1280x1280.jpg

Product 387
https://www.globus-baumarkt.de/media/image/4f/cf/33/4014809114598_produkt_1280x1280.jpg
https://www.globus-baumarkt.de/media/image/bb/7a/95/4014809114598_produkt1_1280x1280.jpg
https://www.globus-baumarkt.de/media/image/48/ef/b5/4014809114598_produkt2_1280x1280.jp

In [244]:
images_df.to_csv('Images-{}'.format(website))

### Updating Product types

In [256]:
df_proc['Product Type Comb'] = np.nan
df_proc['Product Type Comb'].loc[0] = ''

In [257]:
for i in df_proc.index:
    df_proc['Product Type Comb'].loc[i] = []
    
    for key_word in keywords['Product Type'].unique():
        if key_word.lower() in df_proc['Combined'].loc[i].lower():
            if any([key_word.lower() in x.lower() for x in df_proc['Product Type Comb'].loc[i]]):
                continue
            
            else:
                df_proc['Product Type Comb'].loc[i].append(key_word)
    df_proc['Product Type Comb'].loc[i] = ', '.join(df_proc['Product Type Comb'].loc[i])

In [258]:
df_proc['Product Type Comb'] = df_proc['Product Type Comb'].apply(lambda x: np.nan if x=='' else x)

In [259]:
wrong_products = ['Befestigungssysteme', 'Zubehör']
df_proc = df_proc[~df_proc['Product Type Comb'].isin(wrong_products)]
df_proc.shape

(400, 102)

In [260]:
df_proc['Product Type'] = df_proc['Product Type Comb']

In [356]:
df_proc['Product Type'] = df_proc['Product Type'].str.split(', ', expand=True)[0]

In [357]:
df_proc['Product Type'].value_counts()

Massivholzplatte         63
Rahmen                   56
Pfosten                  36
Profilholz               33
Diele                    30
Terrassendiele           23
Glattkantbrett           22
Latte                    22
Konstruktionsvollholz    21
Leimholz                 14
Brett                    12
Parkett                  10
Brett mit Rinde           7
Tischplatte               7
Unterkonstruktion         5
Arbeitsplatte             5
Rauspund                  4
Holzfliese                4
UK-Latte                  3
Massivholzdiele           3
Schalbrett                3
Balken                    2
Universaldiele            2
Rhombus                   1
Stülpschalung             1
Konstruktionsholz         1
Leiste                    1
Name: Product Type, dtype: int64

### Checking Products

In [262]:
df_proc[(df_proc['Product Type']=='') | (df_proc['Product Type'].isna())].shape

(8, 102)

In [263]:
df_proc[df_proc['Product Type'].isna()]['Title'].value_counts()

Massivholzschwelle Douglasie              4
Kosche Ausgleichs- und Unterlagshölzer    1
Kosche Holzschutzunterlage                1
TrendLine WPC Fliese grau                 1
Osmo Befestigungsclip                     1
Name: Title, dtype: int64

### Matching keywords

In [264]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [265]:
main_cols = ['URL','SKU','Product Type','Wood Type','Surface Treatment',
             'Drying Method','Sorting Class','Quality','Combined', 'Surface Quality']

In [266]:
df_proc['Matched Keywords'] = np.nan
df_proc['Other Keywords'] = np.nan
df_proc = df_proc.fillna('-777')
keywords = keywords.fillna('-999')

In [267]:
%%time
def get_keywords(row):
    row['Matched Keywords'] = {}
    row['Other Keywords'] = {}
    
    matched_cols=[]
    for col in ['Product Type','Wood Type','Surface Treatment','Surface Quality',
                'Drying Method','Sorting Class','Quality']:
        
        for key_word in keywords[col].unique():
            if fuzz.token_set_ratio(str(row[col]).lower(), key_word.lower()) > 90 or key_word.lower() in str(row[col]):
                row['Matched Keywords'][row[col]] = key_word
                matched_cols.append(col)
    
    if len(matched_cols)==0:
        row['Other Keywords'] = np.nan
        row['Matched Keywords'] = np.nan
    
    for col in ['Product Type','Wood Type','Surface Treatment','Surface Quality',
                'Drying Method','Sorting Class','Quality']:
        
        if col not in matched_cols and len(matched_cols)>0 and row[col]!=-999:
            row['Other Keywords'][col] = row[col]
    
    return row
df_proc = df_proc.apply(get_keywords, axis=1)

CPU times: user 8.76 s, sys: 464 ms, total: 9.22 s
Wall time: 10.4 s


In [268]:
df_proc[['Matched Keywords','Other Keywords']].isna().sum()/df_proc.shape[0]*100

Matched Keywords    1.0
Other Keywords      1.0
dtype: float64

### Cleaning NaNs



In [269]:
for col in df_proc.columns:
    df_proc[col] =  df_proc[col].apply(lambda x: np.nan if x =='' or x == 'nan' or x == ' ' or x == '-777' or x == '-999' else x)

In [270]:
df_proc[df_proc['Matched Keywords'].isna()].shape

(4, 104)

In [271]:
df_proc[(df_proc['Matched Keywords'].isna()) & 
          (df_proc['Wood Type']=='Holz')].shape

(1, 104)

### Pre-cleaning

In [272]:
for col in df_proc.columns:
    df_proc[col] = df_proc[col].apply(lambda x: np.nan if x == '' or x == ' ' or x=='-999' or x=='-777' or x=='nan'else x)

# GS

In [273]:
import json
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from df2gspread import df2gspread as d2g
scope = ['https://spreadsheets.google.com/feeds'] 
credentials = ServiceAccountCredentials.from_json_keyfile_name('/Users/macbook/Desktop/UpWork/gs-credentials.json', scope) 
gc = gspread.authorize(credentials)
spreadsheet_key = '16TXP67RtrSjgYkMyv0aCkF4VnMB-Wxu9-rGlf_2FCOw'

In [358]:
sheets_clean = df_proc.copy()
sheets_clean.shape

(398, 105)

In [359]:
sheets_clean['Website Name'] = 'www.globus-baumarkt.de'

In [360]:
images_df.drop_duplicates('URL').shape

(400, 2)

In [361]:
sheets_clean=pd.merge(sheets_clean, images_df.drop_duplicates('URL'), on='URL', how='left')

sheets_clean.shape[0], sheets_clean['Images'].isna().sum()

(398, 0)

In [362]:
sheets_clean.head(1)

,Länge,Breite,Stärke,Farbe,Länge-1,Breite-2,Stärke-3,Farbe-4,Unknown,Unknown-5,Unknown-6,Title,SubTitle,Price Item,Price Meta Text,Price Meta,SKU,URL,Option Type,Holzart,Holzart-4,Oberflächenbehandlung,Oberfläche,Höhe,Farbe-2,Höhe-3,Unknown-7,Einsatz,Material,Setz bestehend aus,Einsatz-1,Material-2,Unknown-3,Setz bestehend aus-4,Unknown-1,Inhalt,Unknown-4,Inhalt-5,Unknown-2,schnelle Montage,Material-4,Lieferumfang,Anwendungsbereich,Dauerhaftigkeit nach DIN Dauerhaftigkeitsklasse,zertifiziert nach,Dauerhaftigskeitsklasse,Dauerhaftigkeitsklasse,Hinweis,Stärke-2,Holzart-3,Sortierung,Packinhalt,Deckfläche,Profilart,Holzart-5,Paketinhalt,Unknown-8,Breite-1,Oberfläche-4,Länge-3,Verlegung,Unknown-9,Maße,Holzfeuchte,Maße-1,Holzfeuchte-3,Vielfälltige Einsatzmöglichkeiten,Kanten,Profil,Dekor,Dekor-4,Optik,Nutzschichtdicke,Fugenbild,Veredelung,Aufbau,Mittellage,Länge-2,Optil,Trägerplatt,Nutzungsklasse,Klick-Verbindung,Maße-3,Größe,Größe-2,Reichweite pro Platte,Combined,Quality,Sorting Class,Meta Wood,Wood Type,Combined Surface Treat,Surface Treatment,Surface Quality,Combined Drying Method,Drying Method,Lange mm,Stärke mm,Breite mm,Price Num,Product Type Comb,Product Type,Matched Keywords,Other Keywords,Comb Quality,Website Name,Images
0,300 mm,300 mm,21 mm,grau,300 mm,300 mm,21 mm,grau,leichte Montage durch Steck-Verbindungssystem,zur Verlegung auf festem Untergrund im Innen- ...,"10 Stück im Karton = 0,90 m²",TrendLine WPC Fliese grau,"300 x 300 mm (L x B), Stärke: 21 mm",38.98 € / m²,"Inhalt: 0,9 m² Stückpreis: 35,08 €)","35,08",780800368,https://www.globus-baumarkt.de/trendline-wpc-f...,Plain,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,300 mm; 300 mm; 21 mm; grau; 300 mm; 300...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,300.0,21.0,300.0,35.08,NaN,NaN,NaN,NaN,NaN,www.globus-baumarkt.de,https://www.globus-baumarkt.de/media/image/fb/...


In [363]:
ren_cols = {'Title':'Product Title',
            'Breite mm':'Width (mm)',
            'Lange mm':'Length (mm)', 
            'Stärke mm':'Thickness (mm)',
            'Price Num':'Price (Euro)'}
sheets_clean = sheets_clean.rename(columns = ren_cols)

cols = ['Website Name', 'URL','Product Title','Product Type', 'Wood Type', 'Width (mm)', 'Length (mm)', 'Thickness (mm)', 
        'Price (Euro)','Quality', 'Surface Treatment','Drying Method', 'Surface Quality',
        'Sorting Class','SKU','Images'
       ]
sheets_clean['Product Title'] = sheets_clean['Product Title'] + ' ' + sheets_clean['SubTitle']
sheets_clean = sheets_clean[cols].rename(columns = ren_cols)

In [364]:
sheets_clean = sheets_clean[~sheets_clean['Product Type'].isin(wrong_products)]
sheets_clean.shape

(398, 16)

In [365]:
sheets_clean = sheets_clean.drop_duplicates(['URL','SKU'])
sheets_clean.shape

(398, 16)

In [366]:
sheets_clean[(sheets_clean['Width (mm)'].isna()) &
             (sheets_clean['Length (mm)'].isna()) &
             (sheets_clean['Thickness (mm)'].isna())].shape[0]/sheets_clean.shape[0]*100

1.256281407035176

In [367]:
d2g.upload(sheets_clean[cols].fillna('-'),
           spreadsheet_key,
           'Globus',
           credentials=credentials,
           col_names=True,
           row_names=False,
           start_cell = 'A1',
           clean=True)

<Worksheet 'Globus' id:1123664737>